In [7]:
# CSV to PARQUET

import pandas as pd
import os

# Diretórios
caminho_raiz = 'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\1200 x 1010'
saida_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\sis'

# Criar o diretório de saída, se não existir
os.makedirs(saida_parquet, exist_ok=True)

# Dicionário para armazenar DataFrames por nome base
dfs_paginas = {}

# Caminhar por todas as subpastas e arquivos
for root, _, files in os.walk(caminho_raiz):
    for arquivo in files:
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(root, arquivo)

            try:
                # Lê o CSV com separador pipe e tipos como string
                df = pd.read_csv(caminho_arquivo, delimiter='|', dtype=str)
            except Exception as e:
                print(f'Erro ao ler {caminho_arquivo}: {e}')
                continue

            # Extrai o nome base (antes de _pagina ou da extensão)
            nome_base = arquivo.split('_pagina')[0].replace('.csv', '')

            if nome_base not in dfs_paginas:
                dfs_paginas[nome_base] = []

            if not df.empty:
                dfs_paginas[nome_base].append(df)
                print(f'Lido: {arquivo} com {df.shape[0]} linhas')

# Concatenar e salvar
for nome_base, lista_dfs in dfs_paginas.items():
    if not lista_dfs:
        print(f'{nome_base} - Nenhum dado encontrado')
        continue

    df_final = pd.concat(lista_dfs, ignore_index=True) if len(lista_dfs) > 1 else lista_dfs[0]
    caminho_saida = os.path.join(saida_parquet, f'{nome_base}.parquet')

    try:
        df_final.to_parquet(caminho_saida, index=False)
        print(f'Salvo: {caminho_saida}')
    except Exception as e:
        print(f'Erro ao salvar {nome_base}.parquet: {e}')

print('Conversão concluída!')


Lido: 1200_x_1010_1.csv com 1000000 linhas
Lido: 1200_x_1010_2.csv com 479805 linhas
Salvo: C:\AFL_GABRIEL\csv_parser\DATA_PARQUET\sis\1200_x_1010_1.parquet
Salvo: C:\AFL_GABRIEL\csv_parser\DATA_PARQUET\sis\1200_x_1010_2.parquet
Conversão concluída!


In [2]:
# XLSX to PARQUET

import pandas as pd
import os
 
# Diretório onde os XLSX estão localizados
caminho_xlsx = 'C:\\AFL_GABRIEL\\csv_parser\\DATA\\gabarito\\1200 x 1010'  
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\gabarito'  
 
# Criar o diretório de saída, se não existir
if not os.path.exists(caminho_parquet):
    os.makedirs(caminho_parquet)
 
# Lista para armazenar DataFrames das páginas
dfs_paginas = {}
 
# Iterar pelos arquivos XLSX no diretório
for arquivo in os.listdir(caminho_xlsx):
    if arquivo.endswith('.xlsx'):
        # Caminho completo do arquivo XLSX
        caminho_xlsx = os.path.join(caminho_xlsx, arquivo)
       
        try:
            # Lê todas as planilhas do arquivo XLSX em um dicionário
            sheets_dict = pd.read_excel(caminho_xlsx, sheet_name=None)
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')
            continue
       
        # Extrai o nome base antes da parte de paginação, se houver
        nome_base = arquivo.split('_pagina')[0]
       
        # Se ainda não tiver uma entrada para este nome_base, cria
        if nome_base not in dfs_paginas:
            dfs_paginas[nome_base] = []
       
        # Itera por todas as páginas do arquivo e adiciona os DataFrames
        for nome_sheet, df in sheets_dict.items():
            if not df.empty:
                dfs_paginas[nome_base].append(df)
                print(f'Lido: {arquivo} - Página: {nome_sheet} com {df.shape[0]} linhas')
 
# Processar e unir os DataFrames por nome_base
for nome_base, lista_dfs in dfs_paginas.items():
    # Se houver mais de uma página, concatena os DataFrames
    if len(lista_dfs) > 1:
        df_final = pd.concat(lista_dfs, ignore_index=True)
        print(f'{nome_base} - Páginas unidas com {df_final.shape[0]} linhas')
    elif lista_dfs:
        df_final = lista_dfs[0]
    else:
        print(f'{nome_base} - Nenhum dado encontrado')
        continue
   
    # Caminho para salvar o arquivo Parquet
    caminho_parquet = os.path.join(caminho_parquet, f'{nome_base}.parquet')
   
    # Tenta salvar em formato Parquet
    try:
        df_final.to_parquet(caminho_parquet, index=False)
        print(f'Salvo: {caminho_parquet}')
    except Exception as e:
        print(f'Erro ao salvar {nome_base}.parquet: {e}')
 
print('Conversão concluída!')

Lido: 1200 x 1010.xlsx - Página: Parte 1 com 1000000 linhas
Lido: 1200 x 1010.xlsx - Página: Parte 2 com 479805 linhas
1200 x 1010.xlsx - Páginas unidas com 1479805 linhas
Salvo: C:\AFL_GABRIEL\csv_parser\DATA_PARQUET\gabarito\1200 x 1010.xlsx.parquet
Conversão concluída!


In [9]:
import pandas as pd
import os

# Caminho onde estão os arquivos Parquet
caminho_parquet = 'C:\\AFL_GABRIEL\\csv_parser\\DATA_PARQUET\\sis'

# Lista para armazenar todos os DataFrames
lista_df = []

# Lê todos os arquivos .parquet da pasta
for arquivo in os.listdir(caminho_parquet):
    if arquivo.endswith('.parquet'):
        caminho_arquivo = os.path.join(caminho_parquet, arquivo)
        try:
            df = pd.read_parquet(caminho_arquivo)
            lista_df.append(df)
            print(f'Lido: {arquivo} com {df.shape[0]} linhas')
        except Exception as e:
            print(f'Erro ao ler {arquivo}: {e}')

# Junta todos os DataFrames em um só
df_total = pd.concat(lista_df, ignore_index=True)

# Agrupa por 'Código rubrica' para contar:
# - total de ocorrências
# - número de descrições distintas
resumo = df_total.groupby('Código rubrica').agg(
    Qtd_total=('Descrição da Rubrica', 'count'),
    Qtd_descricoes_distintas=('Descrição da Rubrica', 'nunique')
).reset_index()

# Copia para o clipboard (Ctrl+V no Excel)
resumo.to_clipboard(index=False, excel=True)
print('Resumo com totais copiado para o clipboard com sucesso!')



Lido: 1200_x_1010_1.parquet com 1000000 linhas
Lido: 1200_x_1010_2.parquet com 479805 linhas
Resumo com totais copiado para o clipboard com sucesso!
